In [1]:
import torch
from torchtext.legacy import data

SEED = 1234

torch.manual_seed(SEED)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

In [2]:
# Create Field objects
PHRASE = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm')
SENTIMENT = data.LabelField()

# Create tuples representing the columns
fields = [
  (None, None), # ignore PhraseId column
  (None, None), # ignore SentenceId column
  ('Phrase', PHRASE),
  ('Sentiment', SENTIMENT)
]

# Load the dataset in tsv format
train_ds, test_ds = data.TabularDataset.splits(
   path = './data',
   train = 'train.tsv',
   test = 'test_mapped.tsv',
   format = 'tsv',
   fields = fields,
   skip_header = True
)

In [3]:
import random

# Create validation dataset; splits 70/30 by default
train_ds, valid_ds = train_ds.split(split_ratio=0.7, stratified=True, strata_field='Sentiment', random_state = random.seed(SEED))

print(f'Number of training examples: {len(train_ds)}')
print(f'Number of validation examples: {len(valid_ds)}')
print(f'Number of testing examples: {len(test_ds)}')

Number of training examples: 109241
Number of validation examples: 46819
Number of testing examples: 65327


In [4]:
# Build vocabulary
PHRASE.build_vocab(train_ds, 
                    vectors = "glove.6B.100d", 
                    unk_init = torch.Tensor.normal_)
SENTIMENT.build_vocab(train_ds)

print(f"Unique tokens in PHRASE vocabulary: {len(PHRASE.vocab)}")
print(f"Unique tokens in SENTIMENT vocabulary: {len(SENTIMENT.vocab)}")

Unique tokens in PHRASE vocabulary: 17044
Unique tokens in SENTIMENT vocabulary: 5


In [5]:
BATCH_SIZE = 64

# Only necessary for GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Creating the iterators
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_ds, valid_ds, test_ds), 
    batch_size = BATCH_SIZE,
    sort = False,
    device = device)

In [6]:
# Build the model
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim, 
                 dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        self.convs = nn.ModuleList([
                                    nn.Conv2d(in_channels = 1, 
                                              out_channels = n_filters, 
                                              kernel_size = (fs, embedding_dim)) 
                                    for fs in filter_sizes
                                    ])
        
        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, text):
        
        #text = [sent len, batch size]
        
        text = text.permute(1, 0)
                
        #text = [batch size, sent len]
        
        embedded = self.embedding(text)
                
        #embedded = [batch size, sent len, emb dim]
        
        embedded = embedded.unsqueeze(1)
        
        #embedded = [batch size, 1, sent len, emb dim]
        
        conved = [F.relu(conv(embedded)).squeeze(3) for conv in self.convs]
            
        #conv_n = [batch size, n_filters, sent len - filter_sizes[n]]
        
        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]
        
        #pooled_n = [batch size, n_filters]
        
        cat = self.dropout(torch.cat(pooled, dim = 1))

        #cat = [batch size, n_filters * len(filter_sizes)]
            
        return self.fc(cat)

In [7]:
INPUT_DIM = len(PHRASE.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 100
FILTER_SIZES = [2,3,4]
OUTPUT_DIM = len(SENTIMENT.vocab)
DROPOUT = 0.5
PAD_IDX = PHRASE.vocab.stoi[PHRASE.pad_token]

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)

In [8]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,796,205 trainable parameters


In [9]:
pretrained_embeddings = PHRASE.vocab.vectors

model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.1117, -0.4966,  0.1631,  ...,  1.2647, -0.2753, -0.1325],
        [-0.8555, -0.7208,  1.3755,  ...,  0.0825, -1.1314,  0.3997],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [ 0.2061,  0.1557,  0.5050,  ..., -0.9561,  0.3256,  0.9500],
        [-0.3552,  0.4732,  0.8660,  ...,  0.2483, -0.0049,  0.8731],
        [-0.1431,  0.0487,  0.0565,  ..., -0.0402, -0.3744,  0.5650]])

In [10]:
UNK_IDX = PHRASE.vocab.stoi[PHRASE.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [11]:
# Train the model
import torch.optim as optim

optimizer = optim.Adam(model.parameters())

criterion = nn.CrossEntropyLoss()

model = model.to(device)
criterion = criterion.to(device)

In [12]:
def categorical_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """
    top_pred = preds.argmax(1, keepdim = True)
    correct = top_pred.eq(y.view_as(top_pred)).sum()
    acc = correct.float() / y.shape[0]
    return acc

In [13]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        predictions = model(batch.Phrase)
        
        loss = criterion(predictions, batch.Sentiment)
        
        acc = categorical_accuracy(predictions, batch.Sentiment)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [14]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            predictions = model(batch.Phrase)
            
            loss = criterion(predictions, batch.Sentiment)
            
            acc = categorical_accuracy(predictions, batch.Sentiment)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [15]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [16]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()
    
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'model_CNN.pt')
    
    print(f'Epoch: {epoch+1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

Epoch: 01 | Epoch Time: 2m 48s
	Train Loss: 1.012 | Train Acc: 59.02%
	 Val. Loss: 0.890 |  Val. Acc: 63.89%
Epoch: 02 | Epoch Time: 4m 52s
	Train Loss: 0.861 | Train Acc: 64.73%
	 Val. Loss: 0.854 |  Val. Acc: 65.30%
Epoch: 03 | Epoch Time: 4m 8s
	Train Loss: 0.799 | Train Acc: 67.40%
	 Val. Loss: 0.849 |  Val. Acc: 65.41%
Epoch: 04 | Epoch Time: 3m 23s
	Train Loss: 0.757 | Train Acc: 69.03%
	 Val. Loss: 0.847 |  Val. Acc: 66.00%
Epoch: 05 | Epoch Time: 3m 19s
	Train Loss: 0.725 | Train Acc: 70.30%
	 Val. Loss: 0.854 |  Val. Acc: 66.10%
Epoch: 06 | Epoch Time: 3m 38s
	Train Loss: 0.697 | Train Acc: 71.66%
	 Val. Loss: 0.863 |  Val. Acc: 66.44%
Epoch: 07 | Epoch Time: 3m 51s
	Train Loss: 0.675 | Train Acc: 72.25%
	 Val. Loss: 0.870 |  Val. Acc: 66.46%
Epoch: 08 | Epoch Time: 4m 7s
	Train Loss: 0.652 | Train Acc: 73.27%
	 Val. Loss: 0.881 |  Val. Acc: 66.52%
Epoch: 09 | Epoch Time: 3m 35s
	Train Loss: 0.637 | Train Acc: 73.72%
	 Val. Loss: 0.912 |  Val. Acc: 66.35%
Epoch: 10 | Epoch Tim